In [1]:
import numpy as np
import pandas as pd
# Basic Imports
import numpy as np
import pandas as pd
from colorama import Fore

# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import KFold




#sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict


df_norm = pd.read_csv('/Users/mathildedeschaud/code/Madeschaud/911-tweet/tweet_911/Data/clean_nodis_124.csv')
df_dis = pd.read_csv('/Users/mathildedeschaud/code/Madeschaud/911-tweet/tweet_911/Data/clean_data.csv')


In [2]:
df_norm['disaster_or_not']= 0
df_dis['disaster_or_not']= 1


In [3]:
df = pd.concat([df_norm, df_dis])

In [4]:
df.head()

,Unnamed: 0,tweet_id,tweet_text,disaster_or_not,tweet_clean,words_per_tweet,Unnamed: 0.1,class_label,disaster_year,country,disaster_type,actionable
0,1270274,2001168103,Now you can put your iPhone to work for you! F...,0,put iphone work find today httpisgdkozh,6,NaN,NaN,NaN,NaN,NaN,NaN
1,1025173,1932831900,@3oh3PFR oh no he didnt! see you in Australia!,0,oh didnt see australia,4,NaN,NaN,NaN,NaN,NaN,NaN
2,6824,1469584314,"@electra126 Uh, blame global warming for that ?",0,uh blame global warming,4,NaN,NaN,NaN,NaN,NaN,NaN
3,1543779,2183846794,Finished a song with Ben today,0,finished song ben today,4,NaN,NaN,NaN,NaN,NaN,NaN
4,704294,2256987111,@couver87 how come that everytime I'm online y...,0,come everytime im online vice versa,6,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.disaster_or_not.value_counts()

disaster_or_not
1    124096
0    124000
Name: count, dtype: int64

In [6]:
df_dis.shape, df.columns


((124096, 12),
 Index(['Unnamed: 0', 'tweet_id', 'tweet_text', 'disaster_or_not',
        'tweet_clean', 'words_per_tweet', 'Unnamed: 0.1', 'class_label',
        'disaster_year', 'country', 'disaster_type', 'actionable'],
       dtype='object'))

In [7]:
df.drop(columns=['Unnamed: 0', 'tweet_id', 'Unnamed: 0.1', 'class_label',
        'disaster_year', 'country', 'disaster_type', 'actionable'], inplace=True)

In [8]:
df.head()

,tweet_text,disaster_or_not,tweet_clean,words_per_tweet
0,Now you can put your iPhone to work for you! F...,0,put iphone work find today httpisgdkozh,6
1,@3oh3PFR oh no he didnt! see you in Australia!,0,oh didnt see australia,4
2,"@electra126 Uh, blame global warming for that ?",0,uh blame global warming,4
3,Finished a song with Ben today,0,finished song ben today,4
4,@couver87 how come that everytime I'm online y...,0,come everytime im online vice versa,6


In [9]:
df.disaster_or_not.value_counts()

disaster_or_not
1    124096
0    124000
Name: count, dtype: int64

In [10]:
df.to_csv('obj2_clean_data.csv')

## import new df

In [17]:
data = pd.read_csv('/Users/mathildedeschaud/code/Madeschaud/911-tweet/tweet_911/Data/obj2_clean_data.csv')

In [18]:

def split_data():
    #identify X,y
    X = data['tweet_clean']
    y = data.actionable

    #split data
    return train_test_split( X, y, test_size=0.30, random_state=42)


def tokenize_data(X_train, X_test):
    # tokenize
    tk =Tokenizer()
    tk.fit_on_texts(X_train)

    vocab_size = len(tk.word_index)
    print(f'There are {vocab_size} different words in your corpus')

    return vocab_size, tk.texts_to_sequences(X_train), tk.texts_to_sequences(X_test)


def initialize_model(vocab_size, embedding_dim):
    # build model
    model = Sequential()
    #embedding
    model.add(Embedding(input_dim=vocab_size+1,output_dim=embedding_dim, mask_zero=True))

    #lstm
    model.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=True)))
    # model.add(Bidirectional(LSTM(64, activation='tanh')))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    #compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Recall', 'Precision'])
    return model

def model_bidirectional_lstm():
    # set params
    max_features =10000
    max_len=300
    embedding_dim=2

    X_train, X_test,y_train, y_test = split_data()
    vocab_size, X_train_token, X_test_token = tokenize_data(X_train, X_test)

    # Pad the inputs
    X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post',maxlen=max_len)
    X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post',maxlen=max_len)

    model = initialize_model(vocab_size, embedding_dim)

    #initialize
    print(Fore.MAGENTA + 'Le Bidirectional LSTM est lancé' + Fore.MAGENTA)
    es = EarlyStopping(patience=10, restore_best_weights=True, monitor='val_precision')


    # Train the model
    checkpoint_path = 'Data/checkpoint/model-{epoch:02d}-{val_accuracy:.2f}.hdf5'
    check = ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True)
    history = model.fit(X_train_pad,
                    y_train, batch_size=32,
                    epochs=1,
                    shuffle=True,
                    validation_split = 0.2, #IMPORTANT éviter le data leakage
                    callbacks = [es, check],
                    verbose = 1)
    # # Evaluate the model
    # loss, accuracy = model.evaluate(X_test_pad, y_test)
    # print(f'Test loss: {loss:.4f}')
    # print(f'Test accuracy: {accuracy:.4f}')


    y_pred = model.predict(X_test_pad) # Make cross validated predictions of entire dataset
    print(classification_report(y_test,(y_pred > 0.5).astype(int))) # Pass predictions and true values to Classification report
    return model
